# AiiDA by example: Computing a band structure


:::{admonition} Learning Objectives
:class: learning-objectives

In this section we will present a complete example of an AiiDA workflow, which defines the sequence of calculations needed to compute the band structure of silicon.

How to setup the input data and the details of the workflow execution will be discussed in subsequent sections.
Here we simply give an initial overview of what it means to run an AiiDA workflow.

:::

## Interacting with AiiDA

AiiDA can be controlled in two ways:

1. Using the `verdi` command line interface (CLI), or `%verdi` magic in Jupyter notebooks.
2. Using the `aiida` Python API

For each project in AiiDA, we set up a *profile*, which defines the connection to the data storage, and other settings.

In [ ]:
from local_module import load_temp_profile

data = load_temp_profile(
    name="bands_workflow",
    add_computer=True,
    add_pw_code=True,
    add_sssp=True,
    add_structure_si=True,
)
data

In [ ]:
%verdi status --no-rmq

Within this profile, we have stored the initial input components for our workflow, including the pseudo-potentials, and the silicon structure:

In [ ]:
%verdi storage info --detailed

We have also set up the compute resource that we will use to run the calculations, and the code (`pw.x`) installed on that computer, which we will use to perform the electronic structure calculations.

Here, we will use our "local" machine to run the computations, but AiiDA can also be used to submit calculations to remote supercomputer schedulers, transporting data between the local machine and the remote computer.

In [ ]:
%verdi computer show local_direct

In [ ]:
%verdi code show pw.x@local_direct

## Utilising a pre-defined workflow

AiiDA plugins can declare workflow plugins, for use within AiiDA.
These are workflows that are pre-defined, and can be used as-is, or as a starting point for your own workflows.

Here we utilise the `quantumespresso.pw.bands` workflow defined by the [`aiida-quantumespresso`](https://aiida-quantumespresso.readthedocs.io) plugin.

In [ ]:
%verdi plugin list aiida.workflows

In [ ]:
%verdi plugin list aiida.workflows quantumespresso.pw.bands

![quantumespresso.pw.bands](_static/bands-workflow.svg){height=250px align=center}

The `quantumespresso.pw.bands` workflow provides a helpful method for setting up the default inputs for a given "protocol", as to how fast/precise the calculation should be.
This provides a "builder" object, which stores all the inputs for the workflow.

In [ ]:
from aiida_quantumespresso.workflows.pw.bands import PwBandsWorkChain

builder = PwBandsWorkChain.get_builder_from_protocol(
    code=data.code, 
    structure=data.structure,
    protocol="fast",
)
builder

## Running the workflow

Workflows can be run in the interpreter using the `run` method, in a blocking manner, which we shall do here.

In [ ]:
from aiida import engine

result = engine.run_get_node(builder)
result

Typically however, long running workflows are executed by using the `submit` method.
This will store the initial state of the workflow in the profile storage, and notify the AiiDA daemon to run the workflow in the background.

The AiiDA daemon can be launched using the `verdi daemon start n` command, with `n` being the number of worker processes to launch.
Each worker can asynchronously handle 1000s of individual calculations, allowing for a high-throughput of workflow submissions.

![daemon illustration](_static/aiida/rabbitmq-sysml.svg){height=200px align=center}

Each workflow and node stored in the AiiDA profile is assigned a unique identifier (a.k.a Primary Key), which can be used to reference them.
The execution of the workflows can be monitored using the `verdi process list` command, which will show the status of all running processes in the profile (or also finished ones with `-a`).

In [ ]:
%verdi process list -a

We can also monitor the progress of individual workflows using the `verdi process status` command, which will show the status of the individual steps of the workflow.

In [ ]:
%verdi process status {result.node.pk}

This work-chain demonstrates how we can build up a complex workflow from a series of individual calculations.
In this case, the workflow is made up of the following steps:

1. The `PwRelaxWorkChain` will run multiple Quantum ESPRESSO `vc-relax` calculations, to make sure that there are no Pulay stresses present in the material and that the requested k-points density is respected in case there is a significant volume change in the material.
2. Once the geometry has been optimized, SeeK-path will be used to primitivize and standardize the structure, as well as find the standard path along which to calculate the band structure.
3. A static calculation (`scf`) is run to calculate the charge density for the structure obtained from SeeK-path.
4. Finally, an NSCF is run to calculate the band structure along the path determined by Seek-path.

We shall also discuss in subsequent sections, how the `PwBaseWorkChain` can identify and recover from known failure modes, such as reaching the wall-time limit of the scheduler, or convergence failures.

## Inspecting the results

Once we the workflow has finished, we can inspect the results using the `verdi process show` command, which will show the results of the workflow, and its "attached" outputs.

In [ ]:
%verdi process show {result.node.pk}

### The provenance graph

As well as storing the inputs and outputs of the workflow, and its composite calculations, AiiDA also stores the links between them, which can be used to reconstruct the provenance graph of the workflow.

This can be visualised using the `verdi node graph generate` command, or using the `Graph` Python API.

In [ ]:
from aiida.tools.visualization import Graph

graph = Graph(graph_attr={"rankdir": "TB", "size": "8!,8!"})
graph.recurse_ancestors(result.node, annotate_links="both")
graph.recurse_descendants(result.node, annotate_links="both")
graph.graphviz

### The output structure

AiiDA's `StructureData` class provides integration with both [ASE](https://wiki.fysik.dtu.dk/ase/), and [Pymatgen](https://pymatgen.org/), which can be used to inspect and visualise the structure.

In [ ]:
pym_structure = result.node.outputs.primitive_structure.get_pymatgen()
pym_structure

In [ ]:
from ase.visualize.plot import plot_atoms

ase_atoms = result.node.outputs.primitive_structure.get_ase()
ax = plot_atoms(ase_atoms)

In [ ]:
from ase.visualize.ngl import NGLDisplay

NGLDisplay(ase_atoms).gui

### The output band structure

Finally, we get to our desired result, the band structure of silicon computed using Quantum ESPRESSO 🎉

In [ ]:
from local_module.bandstructure import plot_bandstructure

bands = result.node.outputs.band_structure
fig, ax = plot_bandstructure(bands)